In [ ]:
# --- 설치 (Colab) ---
!pip -q install konlpy mecab-python3 pandas scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.8/588.8 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 30.8 MB/s eta 0:00:00
[알림] Mecab 사용 불가 → Okt로 진행합니다.
사용 형태소 분석기: okt
토큰 예시: ['오늘', '오늘', '오지', '오지', '축구', '축구', '경기', '경기', '예정', '예정', '진행', '진행', '돼다']


,num_sent,num_tokens,vocab_size,avg_len,max_len,ttr
0,7,99,57,14.142857,18,0.5758


In [4]:

# --- 형태소 분석기 준비 (Mecab 우선, 실패 시 Okt) ---
from konlpy.tag import Okt
tokenizer_name = "okt"
okt = Okt()
mecab = None
try:
    from konlpy.tag import Mecab
    mecab = Mecab()
    tokenizer_name = "mecab"
except Exception as e:
    print("[알림] Mecab 사용 불가 → Okt로 진행합니다.")
print("사용 형태소 분석기:", tokenizer_name)

# --- 데이터 (샘플 문장) ---
SENTS = [
    "오늘은 비가 오지만, 축구 경기는 예정대로 진행됐다.",
    "한글 토큰화는 조사와 어미 때문에 영어보다 까다롭다.",
    "메캅을 쓰면 빠르고 정확하지만, 설치가 어렵다는 의견도 있다.",
    "신조어와 외래어가 많은 SNS 텍스트는 전처리가 중요하다.",
    "형태소 분석은 품사 정보를 함께 얻을 수 있어 텍스트 마이닝에 유리하다.",
    "띄어쓰기 오류가 잦은 데이터는 규칙 보정이나 서브워드가 도움이 된다.",
    "도메인에 따라 최적 토크나이저가 달라지므로 비교 실험이 필요하다.",
]

# --- 불용어 & 품사 가중치 ---
STOPWORDS = set("은 는 이 가 을 를 에 에서 으로 도 와 과 의 로 그리고 그러나 하지만 또한 등 또 또는".split())
POS_WEIGHT = {  # 필요한 품사에 가중치(의미 토큰 강조)
    "N": 1.3,   # 명사류(NNG, NNP 등)
    "V": 1.2,   # 동사/형용사류(VV, VA 등)
    "M": 1.0,   # 부사(MAG)
    "J": 0.5,   # 조사(J*)
    "E": 0.5,   # 어미(E*)
    "S": 0.8,   # 기호(SF 등)
}

def morph_tokens_with_pos(text, stem=True):
    if tokenizer_name == "mecab":
        morphs = mecab.pos(text)
    else:
        morphs = okt.pos(text, stem=stem)
    # 불용어 제거 + 품사 접두 가중치 적용(토큰 반복)
    out = []
    for tok, pos in morphs:
        if tok in STOPWORDS or len(tok) < 2:
            continue
        k = POS_WEIGHT.get(pos[:1], 1.0)  # 품사 첫 글자 기준
        out.extend([tok] * int(round(k)))
    return out

# --- 지표: 평균 길이, 어휘 수, TTR ---
import numpy as np, pandas as pd
def corpus_stats(sent_list, tok_fn):
    all_toks, lens = [], []
    for s in sent_list:
        toks = tok_fn(s)
        all_toks.extend(toks)
        lens.append(len(toks))
    vocab = set(all_toks)
    ttr = len(vocab) / max(1, len(all_toks))
    return {
        "num_sent": len(sent_list),
        "num_tokens": len(all_toks),
        "vocab_size": len(vocab),
        "avg_len": float(np.mean(lens)),
        "max_len": int(np.max(lens)),
        "ttr": round(ttr, 4),
    }

print("토큰 예시:", morph_tokens_with_pos(SENTS[3])[:20])
pd.DataFrame([corpus_stats(SENTS, morph_tokens_with_pos)])


사용 형태소 분석기: mecab
토큰 예시: ['신조어', '외래어', 'SNS', '텍스트', '전처리', '중요']


,num_sent,num_tokens,vocab_size,avg_len,max_len,ttr
0,7,49,48,7.0,9,0.9796


In [5]:
print("토큰 예시:", morph_tokens_with_pos(SENTS[1])[:20])
pd.DataFrame([corpus_stats(SENTS, morph_tokens_with_pos)])

토큰 예시: ['한글', '토큰', '조사', '어미', '때문', '영어', '까다롭']


,num_sent,num_tokens,vocab_size,avg_len,max_len,ttr
0,7,49,48,7.0,9,0.9796
